# MPS and MPO operations with ITensor

In [ ]:
using ITensors
using Plots
using HDF5

Create some MPSs

In [ ]:
# Create MPS from ITensor or from Julia array
i = Index(2)
j = Index(2)
k = Index(2)
l = Index(2)
m = Index(2)

cutoff = 1E-20
maxdim = 10
T = randomITensor(i,j,k,l,m) # From ITensor
#T = randn(2^5); # From Julia array
ψ0 = MPS(T,(i,j,k,l,m);cutoff=cutoff,maxdim=maxdim)

ψ0cont = contract(ψ0); # Recover (reshaped) entry matrix

@show ψ0;

In [ ]:
N = 10; # Number of sites
s = siteinds("Qubit", N); # Indices for all sites
#s = siteinds("Qubit", N, conserve_qns = true); 
#s = siteinds("Qubit", N, conserve_qns = false);

In [ ]:
# Create by hand MPS with desired order

ψ1_list = ["0" for n=1:N]; # Magnetization 1 (up) everywhere
ψ1 = MPS(s,ψ1_list);

ψ2 = randomMPS(s,linkdims=10);
ψ2 = orthogonalize!(ψ2, N);

ψ3_list = [isodd(n) ? "1" : "0" for n in 1:N]; # Magnetization 1 (up) and -1 (down) interleaved
ψ3 = MPS(ComplexF64, s, ψ3_list);

@show ψ3_list;

In [ ]:
@show maxlinkdim(ψ1);
@show maxlinkdim(ψ2);
@show maxlinkdim(ψ3);

In [ ]:
@show norm(ψ1), norm(ψ2), norm(ψ3);

Addition of MPSs

In [ ]:
ψ4 = ψ2 + ψ3; # A cutoff of 1e-15 is used by default
ψ5 = +(ψ1, ψ3; cutoff = 1e-8); # Set the cutoff

In [ ]:
@show norm(ψ4), norm(ψ5);

In [ ]:
normalize!(ψ4); # Use ! so state is replaced by normalized one (in-place operation, where memory of output is preallocated)
ψ5 = normalize(ψ5); # Or set A = normalize(B)

In [ ]:
@show linkdims(ψ2);
@show linkdims(ψ3);
@show linkdims(ψ4);

In [ ]:
@show linkdims(ψ1);
@show linkdims(ψ3);
@show linkdims(ψ5);

Truncate states

In [ ]:
ψ4 = truncate(ψ4, cutoff = 1e-1);
@show linkdims(ψ4);

In [ ]:
truncate!(ψ5, maxdim = 1); # Check what happens when maxdim = 1!
@show linkdims(ψ5);

In [ ]:
# Adding a state with itself

ψ11 = ψ1 + ψ1;
@show linkdims(ψ11); # Automatically truncated
@show norm(ψ11); # But not normalized
normalize!(ψ11);

Overlap of MPSs

In [ ]:
dot(ψ1,ψ3)

In [ ]:
inner(ψ5,ψ3) # Check the impact when ψ5 was truncated with maxdim = 1

In [ ]:
dot(ψ1,ψ11)

Apply single-site operator to an MPS

In [ ]:
site = 3;
@show ψ1[site];

In [ ]:
ψ1 = apply(op("H",s[site]),ψ1); # Use Hadamard gate, which creates equal superposition of 0 and 1 
@show ψ1[site];

Apply two-site operator to an MPS

In [ ]:
# First define two-site operator
site1 = 2; site2 = 3;

Oper = op("SWAP",s[site1],s[site2]);
#Oper = op("S-",s[site1])*op("S+",s[site2]);

In [ ]:
orthogonalize!(ψ1,site1) # Set orthogonality center between sites 2 and 3, so other sites are correctly normalized and can be left out

wf = (ψ1[site1] * ψ1[site2]) * Oper # Multiply operator with MPS
noprime!(wf) # Remove primes of result

# Truncate after product to recover MPS structure
inds3 = uniqueinds(ψ1[site1],ψ1[site2]) # Finds indices unique to ψ1[site1]
U,S,V = svd(wf,inds3,cutoff=1E-12) # These indices will go into U when the SVD is done
ψ1[site1] = U;
ψ1[site2] = S*V;

Hadamard product

In [ ]:
# Define two functions and their product
N = 7; # Number of sites
s = siteinds("Qubit", N); # Indices for all sites
xs = range(0, 2pi, length=2^N); # Sampling grid points

f1(x) = sin(x); # First function
ys1 = f1.(xs); # Construction f1(x)

f2(x) = x*exp(0.25*x); # Second function
ys2 = f2.(xs); # Construction f2(x)

f3(x) = f1(x)*f2(x);
ys3 = f3.(xs); # Construction f3(x)

plot(xs, ys3) # Plotting the original function

Encode the functions and their product as MPS

In [ ]:
cutoff = 1E-16;
χ = 30;

ψ1 = MPS(ys1, s, cutoff=cutoff, maxdim=χ);
ψ2 = MPS(ys2, s, cutoff=cutoff, maxdim=χ);
ψ3 = MPS(ys3, s, cutoff=cutoff, maxdim=χ);

@show linkdims(ψ1);
@show linkdims(ψ2);
@show linkdims(ψ3);

Perform Hadamard product of the MPS of the two functions

In [ ]:
had = MPO(N)
had .= convert(MPO, ψ1)
for i = 1:N
    had[i] *= δ(s[i], s[i]', s[i]'')
end

had = replaceprime(had, 2 => 0)

ψ3_Had = apply(had, ψ2; maxdim=maxdim)

@show linkdims(ψ3_Had);

Contract the resulting MPS and plot the result

In [ ]:
Big_ψ = contract(ψ3_Had) # Recovering the original big tensor
J_recon = Array(Big_ψ, s) # Changing the format from ITensor to Julia array for plotting
ys_recon = reshape(J_recon,2^N,1) # Reshaping from N-dimentional tensor to regular array
plot(xs, ys_recon) # Plotting the reconstructed function

Exercise finished!!!